In [44]:
from typing import *
from collections import namedtuple, deque
from ipythonblocks import BlockGrid
import numpy as np

In [282]:
Action  = TypeVar('Action')
Percept = TypeVar('Percept')
SimpleState = namedtuple('SimpleState', ['agent', 'walls', 'dest'])

class Walls:
    def __init__(self, shape, borders = True):
        self.shape = shape
        self.walls = set()
        
    def add(self, pos1, pos2 = None):
        if pos2 == None:
            pos1, pos2 = pos1
        self.walls.add((pos1, pos2))
        
    def __contains__(self, pair):
        pos1, pos2 = pair
        if pos1 == None:
            return pos2[0] == 0 or pos2[1] == 0
        if pos2 == None:
            return pos2[0] == shape[0] - 1 or pos2[1] == shape[1] - 1
        return pair in self.walls

class Robot1D:
    def __init__(self, initial, program: Callable[[Percept],Action]):
        self.position = initial
        self.program  = program
        self.collision = False
        self.sated = False
        self.is_alive = True
    
    def forward(self):
        self.position = (self.position[0]+1, self.position[1])

class SimpleEnvironment:
    def __init__(self, walls, dest, size, agent):
        self.walls = walls
        self.dest  = dest
        self.shape = size
        self.agent = agent
        
    def state(self) -> SimpleState:
        return SimpleState(self.agent.position, self.walls, self.dest)
    
    def percept(self) -> Percept:
        return self.agent.position == self.dest
    
    def execute(self, action: Action):
        if action == 'forward':
            px, py = self.agent.position
            if px >= self.shape[0] - 1:
                self.agent.is_alive = False
            else:
                self.agent.forward()
        if action == 'nothing':
            pass
    
    def is_done(self):
        return not self.agent.is_alive
    
    def step(self) -> SimpleState:
        if self.is_done():
            raise Exception()
        self.execute(self.agent.program(self.percept()))
        return self.state()
        
    def run(self, n = 1000) -> Iterator[SimpleState]:
        c = 0
        while not self.is_done() and c < n:
            c += 1
            yield self.step()
            
def _draw_state(shape, grid: BlockGrid, state: SimpleState):
    agent_color = (255, 0, 0)
    dest_color = (0, 255, 0)
    wall_color = (19,19,19)
    nowall_color = (238, 238, 238)
    empty_color = (255, 255, 255)
    n, m = shape
    gn, gm = grid.shape
    agent, walls, dest = state
    
    for i in range(gn):
        grid[0,i].set_colors(*wall_color)
        grid[0,i].size = 2
        grid[gn-1,i].set_colors(*wall_color)
        grid[gn-1,i].size = 2
    for j in range(gm):
        grid[j,0].set_colors(*wall_color)
        grid[j,0].size = 2
        grid[j,gm-1].set_colors(*wall_color)
        grid[j,gm-1].size = 2
    
    for i in range(n-1):
        for j in range(m-1):
            positions = (i,j), (i+1,j), (i,j+1), (i+1,j+1)
            pairs = ( (positions[0],positions[1])
                    , (positions[0], positions[2])
                    , (positions[1], positions[3])
                    , (positions[2], positions[3])
                    )
            for p in positions:
                col = agent_color if p == agent \
                        else dest_color if p == dest \
                        else empty_color
                u, v = p
                grid[u*2+1, v*2+1].set_colors(*col)
            for pair in pairs:
                u, v = pair[0]
                du, dv = pair[1][0] - pair[0][0], pair[1][1] - pair[0][1]
                col = (wall_color if pair in walls else nowall_color)
                block = grid[u*2+1+du, v*2+1+dv]
                block.set_colors(*col)
                block.size = 2
            block = grid[i*2+2, j*2+2]
            block.set_colors(*nowall_color)
            block.size = 2

    
def draw_states(env: SimpleEnvironment, states: Iterable[SimpleState]):
    n,m = env.shape
    grid = BlockGrid(2*n+1, 2*m+1)
    
    for state in states:
        _draw_state(env.shape, grid, state)
        grid.flash()
    grid.show()

In [284]:
walls = Walls(10,10)
for j in range(0,9):
    for i in range(1-(j%2),10-(j%2)):
        walls.add((i,j),(i,j+1))
        
env = SimpleEnvironment( walls
                       , (9,9)
                       , (10,10)
                       , Robot1D((8,0), lambda sated: 'nothing' if sated else 'forward')
                       )
draw_states(env, env.run(10))

,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,


In [252]:
walls.walls

{((1, 0), (1, 1))}